# 기능
1. 비디오 파일에서 프레임을 추출합니다.
    - ffmepg 명령 사용

In [1]:
import re
import os
import subprocess
from pathlib import Path

import cv2
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# root directory
dataset_dir = Path("/home/jongphago/project/ultralytics/datasets")
data_csv_dir = "aihub/sample/videos.csv"
sub_dir = "aihub/sample/원천데이터/RGB"
out_dir = "aihub/sample/images"
scenario_id_pattern = r"(?<=시나리오)\d{2}"
camera_id_pattern = r"(?<=카메라)\d{2}"
fps = 1

In [3]:
# Data DataFrame (video name table)
data_df = pd.read_csv(dataset_dir / data_csv_dir)

In [4]:
# __init__
# IDs: video, camera, scenario & task
for rows in data_df.itertuples():
    # data frame single row data
    task = rows.task
    video_dir = rows.video_dir + ".avi"
    scenario_id = re.findall(scenario_id_pattern, video_dir)[0]
    camera_id = re.findall(camera_id_pattern, video_dir)[0]

    # frame directory
    frame_dir = f"aihub/sample/frames/scenario{scenario_id}/camera{camera_id}"
    os.makedirs(dataset_dir / frame_dir, exist_ok=True)

    # video_path
    video_path = str(dataset_dir / sub_dir / video_dir)
    assert os.path.exists(video_path)

In [5]:
# update FPS
if fps is None:
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

In [ ]:
cap = cv2.VideoCapture(video_path)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    

In [ ]:
plt.imshow(frame)

In [ ]:
cv2.imwrite("a.png", frame)

In [ ]:
# run: ffmpeg 명령어 실행
subprocess.run(
    [
        "ffmpeg",
        *("-i", video_path),
        *("-r", str(fps)),
        f"{dataset_dir / frame_dir}/s{scenario_id}_c{camera_id}_f%04d.png",
    ]
)

print("프로세스가 성공적으로 종료되었습니다.")

In [ ]:
# link: Make symbolic link
for directory, _, file_names in os.walk(dataset_dir / frame_dir):
    root = Path(directory)
    srcs = []
    for file_name in file_names:
        srcs.append(root / file_name)
        
images_dir = dataset_dir / out_dir / task
os.makedirs(images_dir, exist_ok=True)
for src in srcs:
    dst = images_dir / src.parts[-1]
    os.symlink(src, dst)

# Video class

In [ ]:
import os
import subprocess
from pathlib import Path

import yaml
import pandas as pd
from box import Box

from codes.config.pattern import find
from codes.config.pattern import camera_id_pattern, scenario_id_pattern

In [ ]:
class Video:
    def __init__(self, video_dir, task):
        self.video_dir = video_dir
        self.task = task


class AIHub(Video):

    def __init__(self, video_dir, task, cfg):
        super().__init__(video_dir, task)
        self.camera_id: str = find(camera_id_pattern, self.video_dir)  # 01
        self.scenario_id: str = find(scenario_id_pattern, self.video_dir)  # 11
        self.fps: int = cfg.fps  # 30
        self.dirs(cfg)

    def dirs(self, cfg):
        self.path = Path(cfg.path)
        self.raw = self.path / cfg.raw  # ex. aihub/sample/원천데이터/RGB
        self.video_dir = (
            self.path / self.raw / self.video_dir
        )  # ex. 시나리오01/카메라07.avi
        self.out = self.path / cfg.out  # ex. aihub/sample/frames
        sub_dir = f"scenario{self.scenario_id}/camera{self.camera_id}"
        self.frames_dir = self.out / sub_dir
        self.img = self.path / cfg.img  # ex. aihub/sample/images
        self.images_dir = self.img / self.task

    def extract(self):
        os.makedirs(self.frames_dir, exist_ok=True)
        subprocess.run(
            [
                "ffmpeg",
                *("-i", self.video_dir),
                *("-r", str(self.fps)),
                f"{self.frames_dir}/s{self.scenario_id}_c{self.camera_id}_f%04d.png",
            ]
        )

    def link(self):
        # link: Make symbolic link
        os.makedirs(self.images_dir, exist_ok=True)
        srcs = []
        for directory, _, file_names in os.walk(self.frames_dir):
            root = Path(directory)
            for file_name in file_names:
                srcs.append(root / file_name)
        for src in srcs:
            dst = self.images_dir / src.parts[-1]
            if not os.path.exists(dst):
                os.symlink(src, dst)

In [ ]:
root = Path("/home/jongphago/project/ultralytics")
config_name = "aihub-sample"
cfg_path = root / f"ultralytics/cfg/datasets/{config_name}.yaml"
with open(cfg_path, "r") as file:
    cfg = Box(yaml.safe_load(file))

In [ ]:
# root directory
dataset_dir = Path(cfg.path)
data_df = pd.read_csv(dataset_dir / cfg.csv)
for rows in data_df.itertuples():
    aihub = AIHub(rows.video_dir, rows.task, cfg)
    aihub.extract()
    aihub.link()